Data Set Link: 
RAF DB: https://www.kaggle.com/datasets/shuvoalok/raf-db-dataset/versions/2

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# ==========================================
# 1. CONFIGURATION
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths provided by you
PRETRAINED_PATH = '/kaggle/input/fer2013/pytorch/default/1/best_fer2013_backbone.pth'
TRAIN_DIR = '/kaggle/input/raf-db-dataset/DATASET/train'
TEST_DIR = '/kaggle/input/raf-db-dataset/DATASET/test'

# Hyperparameters
BATCH_SIZE = 64
LEARNING_RATE = 1e-4  # Standard LR for fine-tuning
EPOCHS = 15           # RAF-DB converges reasonably fast

# RAF-DB Mapping (Ref: User provided)
# 1:Surprise, 2:Fear, 3:Disgust, 4:Happiness, 5:Sadness, 6:Anger, 7:Neutral
# Note: ImageFolder will map folder "1" to index 0, "2" to index 1, etc.
# We will just verify this mapping in the print statements.

# ==========================================
# 2. DATA AUGMENTATION (Crucial for "Real World" Data)
# ==========================================
# We use stronger augmentation here than FER2013 because RAF-DB is color and high-res
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Important for webcam variance
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ==========================================
# 3. LOAD DATA
# ==========================================
print("Loading Datasets...")
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
val_dataset = datasets.ImageFolder(TEST_DIR, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Classes found: {train_dataset.classes}")
print(f"Example: Folder '{train_dataset.classes[0]}' mapped to Index 0")

# ==========================================
# 4. MODEL SETUP (The Intelligent Transfer)
# ==========================================
def load_model_safely(pretrained_path):
    print(f"\nInitializing ResNet18...")
    model = models.resnet18()
    
    # 1. Match the architecture of the SAVED model (FER2013 had 7 classes)
    model.fc = nn.Linear(model.fc.in_features, 7)
    
    # 2. Load the weights
    if os.path.exists(pretrained_path):
        print(f"Loading weights from: {pretrained_path}")
        try:
            state_dict = torch.load(pretrained_path, map_location=DEVICE)
            model.load_state_dict(state_dict)
            print(">> FER2013 weights loaded successfully!")
        except Exception as e:
            print(f"!! Error loading weights: {e}")
            print("!! Starting from scratch (ImageNet weights) instead.")
            model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    else:
        print("!! Pretrained path not found. Using ImageNet weights.")
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        
    # 3. RESET THE HEAD
    # The saved 'fc' layer knows FER2013 mapping (0=Angry).
    # We need to learn RAF-DB mapping (0=Surprise).
    # We re-initialize this layer to random weights.
    print("Resetting final classification layer for RAF-DB...")
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 7) # New random weights for 7 RAF-DB classes
    
    return model.to(DEVICE)

model = load_model_safely(PRETRAINED_PATH)

# ==========================================
# 5. TRAINING LOOP
# ==========================================
criterion = nn.CrossEntropyLoss()
# Using a slightly lower LR for the backbone, but we update ALL parameters
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

def train_one_epoch(epoch_index):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
    avg_loss = running_loss / len(train_loader)
    acc = 100 * correct / total
    return avg_loss, acc

def validate():
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
    avg_loss = running_loss / len(val_loader)
    acc = 100 * correct / total
    return avg_loss, acc

# ==========================================
# 6. EXECUTION
# ==========================================
best_acc = 0.0
print("\nStarting RAF-DB Training...")

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(epoch)
    val_loss, val_acc = validate()
    
    print(f"Epoch {epoch}/{EPOCHS}:")
    print(f"  Train -> Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%")
    print(f"  Valid -> Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%")
    
    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_rafdb_model.pth')
        print(f"  >>> New Best Model Saved (Acc: {best_acc:.2f}%)")

print("\nTraining Complete.")

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt

# ==========================================
# 1. SETUP
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = 'best_rafdb_model.pth'

# Define the standard transforms (Validation version - no augmentation)
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# RAF-DB Raw Classes (Mapped by Folder Name 1-7)
# Note: ImageFolder sorts folders 1-7, so Index 0 is Folder '1', etc.
RAW_CLASSES = {
    0: 'Surprise',
    1: 'Fear',
    2: 'Disgust',
    3: 'Happiness',
    4: 'Sadness',
    5: 'Anger',
    6: 'Neutral'
}

# ==========================================
# 2. LOAD THE MODEL
# ==========================================
def load_inference_model():
    model = models.resnet18()
    # We must match the structure (7 classes) to load weights
    model.fc = torch.nn.Linear(model.fc.in_features, 7)
    
    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
        print("Model loaded successfully!")
    except FileNotFoundError:
        print(f"Error: {MODEL_PATH} not found. Please train the model first.")
        return None
        
    model = model.to(DEVICE)
    model.eval() # Set to evaluation mode (freezes BatchNorm/Dropout)
    return model

model = load_inference_model()

# ==========================================
# 3. THE LOGIC LAYER (7 Classes -> 4 Interview Labels)
# ==========================================
def get_interview_prediction(probs):
    """
    Manually sums probabilities to create Interview Categories.
    Input: Tensor of 7 probabilities [Surprise, Fear, Disgust, Happy, Sad, Anger, Neutral]
    """
    p = probs.tolist()[0] # Convert to standard list
    
    # 1. Stressed = Fear(1) + Sadness(4) + Surprise(0)
    # We include Surprise because in interviews, shock often looks like stress.
    score_stressed = p[1] + p[4] + p[0]
    
    # 2. Unprofessional = Disgust(2) + Anger(5)
    score_unprofessional = p[2] + p[5]
    
    # 3. Confident = Happiness(3)
    score_confident = p[3]
    
    # 4. Composed = Neutral(6)
    score_composed = p[6]
    
    scores = {
        "Stressed": score_stressed,
        "Unprofessional": score_unprofessional,
        "Confident": score_confident,
        "Composed": score_composed
    }
    
    # Return the category with the highest score
    best_category = max(scores, key=scores.get)
    return best_category, scores

# ==========================================
# 4. PREDICTION FUNCTION
# ==========================================
def predict_emotion(image_path):
    if model is None: return
    
    try:
        # Load and Preprocess
        img_raw = Image.open(image_path).convert('RGB')
        img_tensor = test_transforms(img_raw).unsqueeze(0).to(DEVICE)
        
        # Forward Pass
        with torch.no_grad():
            outputs = model(img_tensor)
            # Apply Softmax to get probabilities (0.0 to 1.0)
            probs = F.softmax(outputs, dim=1)
        
        # Get Raw RAF-DB Result
        raw_score, raw_idx = torch.max(probs, 1)
        raw_emotion = RAW_CLASSES[raw_idx.item()]
        
        # Get Interview Logic Result
        interview_cat, interview_scores = get_interview_prediction(probs)
        
        # Visualization
        plt.figure(figsize=(10, 4))
        
        # Plot Image
        plt.subplot(1, 2, 1)
        plt.imshow(img_raw)
        plt.title(f"Input Image\nRaw: {raw_emotion} ({raw_score.item():.2f})")
        plt.axis('off')
        
        # Plot Interview Scores
        plt.subplot(1, 2, 2)
        cats = list(interview_scores.keys())
        vals = list(interview_scores.values())
        colors = ['orange', 'red', 'green', 'blue'] # Stressed, Unprof, Confident, Composed
        
        plt.bar(cats, vals, color=colors)
        plt.title(f"Interview Coach Diagnosis: {interview_cat}")
        plt.ylim(0, 1.0)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        plt.show()
        
        print(f"Detailed Scores:")
        for cat, score in interview_scores.items():
            print(f"  {cat}: {score*100:.1f}%")

    except Exception as e:
        print(f"Error processing image: {e}")

# ==========================================
# 5. TEST IT HERE
# ==========================================
# Replace this path with any image from the RAF-DB Test folder 
# OR upload your own photo to Kaggle and copy the path.
# Example path: '/kaggle/input/raf-db-dataset/DATASET/Test/1/test_0001.jpg'

test_image_path = '/kaggle/input/raf-db-dataset/DATASET/test/5/test_0031_aligned.jpg' # Update this!

# Check if file exists before running to avoid error spam
import os
if os.path.exists(test_image_path):
    predict_emotion(test_image_path)
else:
    print(f"Please update 'test_image_path' to a valid file path.")

In [ ]:
!pip install mediapipe

In [ ]:
import os

# Download the BlazeFace model if you haven't already
MODEL_PATH = 'blaze_face_short_range.tflite'
if not os.path.exists(MODEL_PATH):
    print(f"Downloading MediaPipe Face Detection model...")
    os.system(f"wget -q -O {MODEL_PATH} https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite")

In [ ]:
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt

# New MediaPipe Imports
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# ==========================================
# 1. SETUP
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = 'best_rafdb_model.pth' # Make sure this matches your saved file name

# Standard transforms
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

RAW_CLASSES = {0:'Surprise', 1:'Fear', 2:'Disgust', 3:'Happiness', 
               4:'Sadness', 5:'Anger', 6:'Neutral'}

# ==========================================
# 2. FACE CROPPER (New Tasks API Version)
# ==========================================
class FaceCropper:
    def __init__(self, model_path='blaze_face_short_range.tflite'):
        # Create FaceDetector options
        base_options = python.BaseOptions(model_asset_path=model_path)
        options = vision.FaceDetectorOptions(base_options=base_options, min_detection_confidence=0.5)
        self.detector = vision.FaceDetector.create_from_options(options)

    def get_crop(self, image_path):
        # Read image with OpenCV
        image = cv2.imread(image_path)
        if image is None: return None
        
        # Convert to RGB (MediaPipe requires RGB)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load into MediaPipe Image format
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)

        # Detect
        detection_result = self.detector.detect(mp_image)
        
        if detection_result.detections:
            # Get the first face
            detection = detection_result.detections[0]
            bbox = detection.bounding_box
            
            # The Tasks API gives absolute pixel coordinates directly
            x = bbox.origin_x
            y = bbox.origin_y
            w = bbox.width
            h = bbox.height
            
            ih, iw, _ = image.shape
            
            # --- Smart Margin Logic (10%) ---
            margin_x = int(w * 0.1)
            margin_y = int(h * 0.1)
            
            x = max(0, x - margin_x)
            y = max(0, y - margin_y)
            w = min(iw - x, w + 2 * margin_x)
            h = min(ih - y, h + 2 * margin_y)
            
            # Crop
            face_crop = image_rgb[y:y+h, x:x+w]
            
            # Fallback if crop is empty
            if face_crop.size == 0:
                return Image.fromarray(image_rgb)
                
            return Image.fromarray(face_crop)
        
        else:
            print("Warning: No face detected. Using full image.")
            return Image.fromarray(image_rgb)

# Initialize Cropper
cropper = FaceCropper()

# ==========================================
# 3. LOAD MODEL & LOGIC
# ==========================================
def load_inference_model():
    model = models.resnet18()
    model.fc = torch.nn.Linear(model.fc.in_features, 7)
    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    except FileNotFoundError:
        print(f"Error: {MODEL_PATH} not found. Please train/upload the model.")
        return None
    model = model.to(DEVICE)
    model.eval() 
    return model

model = load_inference_model()

def get_interview_prediction(probs):
    p = probs.tolist()[0]
    scores = {
        "Stressed": p[1] + p[4] + p[0],       # Fear + Sad + Surprise
        "Unprofessional": p[2] + p[5],        # Disgust + Anger
        "Confident": p[3],                    # Happy
        "Composed": p[6]                      # Neutral
    }
    return max(scores, key=scores.get), scores

# ==========================================
# 4. PREDICT FUNCTION
# ==========================================
def predict_emotion(image_path):
    if model is None: return

    # A. Get Crop
    face_img = cropper.get_crop(image_path)
    if face_img is None:
        print("Error: Could not read image file.")
        return

    # B. Predict
    img_tensor = test_transforms(face_img).unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = F.softmax(outputs, dim=1)
    
    raw_idx = torch.max(probs, 1)[1].item()
    interview_cat, interview_scores = get_interview_prediction(probs)
    
    # C. Visualize
    original_img = Image.open(image_path)
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.imshow(original_img)
    plt.title("Original Input")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(face_img)
    plt.title(f"MediaPipe Crop\nRaw: {RAW_CLASSES[raw_idx]}")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    cats = list(interview_scores.keys())
    vals = list(interview_scores.values())
    colors = ['orange', 'red', 'green', 'blue']
    plt.bar(cats, vals, color=colors)
    plt.title(f"Result: {interview_cat}")
    plt.ylim(0, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

# ==========================================
# 5. TEST
# ==========================================
# Update this path
test_image_path = '/kaggle/input/myphotos/WIN_20260131_14_49_36_Pro.jpg' 

if os.path.exists(test_image_path):
    predict_emotion(test_image_path)
else:
    print(f"Please update 'test_image_path' to a valid file path.")